In [1]:
!pip install ebaysdk

In [19]:
from ebaysdk.finding import Connection as Finding
from ebaysdk.merchandising import Connection as Merch
from ebaysdk.exception import ConnectionError
import pandas as pd
import time

In [20]:
API_KEY = 'BoYang-find-PRD-1e312bc95-0039821d'

In [21]:
# Creating a Python dictionary with simplified keys for each category description
christmas_categories = {
    "holiday_decor": "117419",
    "christmas_trees": "117406",
    "christmas_lights": "117422",
    "ornaments": "117419",
    "wreaths_garlands": "117421",
    "crackers": "117420",
    "stockings_hangers": "117423",
    "figurines": "117424",
    "table_decor": "117425",
    "yard_decor": "117426",
    "gift_boxes": "102380",
    "gift_tags_stickers": "184488",
    "nativity_items": "156862",
    "christmas_costumes": "53361",
    "christmas_music": "11233",
    "christmas_movies": "11232",
    "christmas_books": "171243",
    "christmas_cards": "170098",
    "christmas_candles": "46782",
    "christmas_crafts": "116724",
    "christmas_cookware": "20628",
    "christmas_tableware": "36027",
    "christmas_bedding": "20444",
    "christmas_clothing": "155241",
    "christmas_jewelry": "10968",
    "christmas_toys": "19028",
    "christmas_games": "233",
    "christmas_electronics": "293",
    "christmas_home_appliances": "20710",
    "christmas_sporting_goods": "888",
    "christmas_health_beauty": "26395",
    "christmas_automotive": "6000",
    "christmas_collectibles": "1",
    "christmas_art": "550",
    "christmas_musical_instruments": "619",
    "christmas_pet_supplies": "1281",
    "christmas_baby_products": "2984",
    "christmas_business_industrial": "12576",
    "christmas_real_estate": "10542",
    "christmas_travel": "3252",
    "christmas_gift_cards": "172008"
}


In [22]:
christmas_list = list(christmas_categories.values())
christmas_list

['117419',
 '117406',
 '117422',
 '117419',
 '117421',
 '117420',
 '117423',
 '117424',
 '117425',
 '117426',
 '102380',
 '184488',
 '156862',
 '53361',
 '11233',
 '11232',
 '171243',
 '170098',
 '46782',
 '116724',
 '20628',
 '36027',
 '20444',
 '155241',
 '10968',
 '19028',
 '233',
 '293',
 '20710',
 '888',
 '26395',
 '6000',
 '1',
 '550',
 '619',
 '1281',
 '2984',
 '12576',
 '10542',
 '3252',
 '172008']

In [24]:
import time
from ebaysdk.exception import ConnectionError

# Parameters
categories = christmas_list
marketplaces = ["EBAY-US", "EBAY-CA"]
keyword = "Christmas"
MAX_ITEMS = 1000000
ENTRIES_PER_PAGE = 100
sleep_time = 1  # Standard wait time between each request
cooldown_time = 300  # Wait time (5 minutes) when rate limit is reached

category_marketplace_data = {}

try:
    for marketplace in marketplaces:
        for category_id in categories:
            print(f"Fetching items for category ID: {category_id} on {marketplace}")
            api = Finding(appid=API_KEY, config_file=None, siteid=marketplace)
            page_number = 1
            total_items_retrieved = 0

            # CSV filename for current marketplace and category
            csv_filename = f"{category_id}_{marketplace}.csv"
            pd.DataFrame().to_csv(csv_filename, index=False)  # Initialize CSV file with headers

            # Start fetching data in batches
            while total_items_retrieved < MAX_ITEMS:
                try:
                    response = api.execute('findItemsAdvanced', {
                        'keywords': keyword,
                        'categoryId': category_id,
                        'paginationInput': {
                            'entriesPerPage': ENTRIES_PER_PAGE,
                            'pageNumber': page_number
                        },
                        'sortOrder': 'BestMatch',
                        'outputSelector': [
                            'SellerInfo', 'StoreInfo', 'PictureURLSuperSize',
                            'PictureURLLarge', 'UnitPriceInfo', 'ConditionHistogram',
                            'AspectHistogram', 'GalleryInfo', 'ShippingInfo', 'ListingInfo',
                            'ReturnsAcceptedOnly'
                        ]
                    })

                    # Check if 'searchResult' and 'item' exist in the response
                    if hasattr(response.reply, 'searchResult') and hasattr(response.reply.searchResult, 'item'):
                        items = response.reply.searchResult.item
                    else:
                        print(f"No more items found for category {category_id} on {marketplace}.")
                        break

                    # Process each item and append to CSV
                    batch_data = []
                    for item in items:
                        item_data = {
                            "Title": getattr(item, 'title', "N/A"),
                            "Item ID": getattr(item, 'itemId', "N/A"),
                            "Price": getattr(getattr(item, 'sellingStatus', {}), 'currentPrice', {}).get('value', "N/A"),
                            "Currency": getattr(getattr(item, 'sellingStatus', {}), 'currentPrice', {}).get('_currencyId', "N/A"),
                            "Condition": getattr(getattr(item, 'condition', {}), 'conditionDisplayName', "N/A"),
                            "Category": getattr(getattr(item, 'primaryCategory', {}), 'categoryName', "N/A"),
                            "Seller": getattr(getattr(item, 'sellerInfo', {}), 'sellerUserName', "N/A"),
                            "Feedback Score": getattr(getattr(item, 'sellerInfo', {}), 'feedbackScore', "N/A"),
                            "Positive Feedback %": getattr(getattr(item, 'sellerInfo', {}), 'positiveFeedbackPercent', "N/A"),
                            "Top Rated Seller": getattr(getattr(item, 'sellerInfo', {}), 'topRatedSeller', "N/A"),
                            "Shipping Type": getattr(getattr(item, 'shippingInfo', {}), 'shippingType', "N/A"),
                            "Shipping Cost": getattr(getattr(getattr(item, 'shippingInfo', {}), 'shippingServiceCost', {}), 'value', "N/A"),
                            "Listing Type": getattr(getattr(item, 'listingInfo', {}), 'listingType', "N/A"),
                            "Buy It Now Available": getattr(getattr(item, 'listingInfo', {}), 'buyItNowAvailable', "N/A"),
                            "Start Time": getattr(getattr(item, 'listingInfo', {}), 'startTime', "N/A"),
                            "End Time": getattr(getattr(item, 'listingInfo', {}), 'endTime', "N/A"),
                            "Store Name": getattr(getattr(item, 'storeInfo', {}), 'storeName', "N/A"),
                            "Store URL": getattr(getattr(item, 'storeInfo', {}), 'storeURL', "N/A"),
                            "Gallery URL": getattr(item, 'galleryURL', "N/A"),
                            "Large Image URL": getattr(item, 'pictureURLLarge', "N/A"),
                            "Super Size Image URL": getattr(item, 'pictureURLSuperSize', "N/A"),
                            "View Item URL": getattr(item, 'viewItemURL', "N/A"),
                            "Marketplace": marketplace
                        }
                        batch_data.append(item_data)
                        total_items_retrieved += 1

                        # Stop if we reached the maximum for this category
                        if total_items_retrieved >= MAX_ITEMS:
                            break

                    # Save batch data to CSV
                    pd.DataFrame(batch_data).to_csv(csv_filename, mode='a', header=(page_number == 1), index=False, encoding='utf-8-sig')
                    print(f"Saved {len(batch_data)} items to {csv_filename}")

                    # Increment page and pause
                    page_number += 1
                    time.sleep(sleep_time)

                except ConnectionError as e:
                    error_response = e.response.dict()
                    error_details = error_response.get('errorMessage', {}).get('error', {})

                    # Check for rate limit error (errorId: 10001)
                    if error_details.get('errorId') == '10001':
                        print("Rate limit exceeded. Pausing for cooldown period.")
                        time.sleep(cooldown_time)  # Wait for the cooldown period
                    else:
                        print("Connection error:", e)
                        print(error_response)
                    break  # Exit the loop for other connection errors

except ConnectionError as e:
    print("Error connecting to eBay API:", e)
    print(e.response.dict())

Fetching items for category ID: 117419 on EBAY-US
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 99 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
S

Saved 99 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 items to 117419_EBAY-US.csv
Saved 100 ite

Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
Saved 14 items to 117426_EBAY-US.csv
S

Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 items to 171243_EBAY-US.csv
Saved 100 it

Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY-US.csv
Saved 56 items to 20628_EBAY

Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY-US.csv
Saved 32 items to 19028_EBAY

Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 100 items to 293_EBAY-US.csv
Saved 75 items to 293_EBAY-US.csv
Saved 19 items to 293_EBAY-US.csv
No more items found for category 293 on EBAY-US.
Fetching items for category ID: 20710 on EBAY-US
Saved 95 items to 20710_EBAY-US.csv
Saved 63 items to 20710_EBAY-US.csv
Saved 52 items to 20710_EBAY-US.csv
Saved 5 items to 20710_EBAY-US.csv
No more items found for category 20710 on EBAY-US.
Fetching i

Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY-US.csv
Saved 100 items to 2984_EBAY

Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
Saved 25 items to 172008_EBAY-US.csv
No more items found for category 172008 on EBAY-US.
Fetching items for category ID: 117419 on EBAY-CA
Connection error: 'findItemsAdvanced: Internal Server Error, Domain: Marketplace, Severity: Error, errorId: 6, Please specify a valid GLOBAL-ID'
{'errorMessage': {'error': {'errorId': '6', 'domain': 'Marketplace', 'severity': 'Error

Connection error: 'findItemsAdvanced: Internal Server Error, Domain: Marketplace, Severity: Error, errorId: 6, Please specify a valid GLOBAL-ID'
{'errorMessage': {'error': {'errorId': '6', 'domain': 'Marketplace', 'severity': 'Error', 'category': 'Application', 'message': 'Please specify a valid GLOBAL-ID', 'subdomain': 'MarketplaceCommon'}}}
Fetching items for category ID: 36027 on EBAY-CA
Connection error: 'findItemsAdvanced: Internal Server Error, Domain: Marketplace, Severity: Error, errorId: 6, Please specify a valid GLOBAL-ID'
{'errorMessage': {'error': {'errorId': '6', 'domain': 'Marketplace', 'severity': 'Error', 'category': 'Application', 'message': 'Please specify a valid GLOBAL-ID', 'subdomain': 'MarketplaceCommon'}}}
Fetching items for category ID: 20444 on EBAY-CA
Connection error: 'findItemsAdvanced: Internal Server Error, Domain: Marketplace, Severity: Error, errorId: 6, Please specify a valid GLOBAL-ID'
{'errorMessage': {'error': {'errorId': '6', 'domain': 'Marketplace'